<a href="https://colab.research.google.com/github/sirius70/NLP_HW2/blob/main/shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interpreting Classifier Weights

In this experiment, you will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

You will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

You will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks is hard enoough.

In [1]:
import json
import requests
import re
import numpy as np
from scipy import sparse
from collections import Counter
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate,LeaveOneOut

In [2]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/dasmiq/cs6120-assignment2/refs/heads/main/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

This is where you will implement two functions to featurize the data:

In [3]:
def tokenize(text):
    """
    Tokenizes text into words that contain at least one alphabetic character.
    Splits on underscores and other non-word characters.
    """
    # Replace underscores with spaces so they're split
    text = text.replace("_", " ")

    # Match tokens with at least one alphabetic character
    return re.findall(r'\b\w*[a-zA-Z]\w*\b', text.lower())

In [4]:
# TODO: Implement createBasicFeatures
# NB: The current contents are for testing only
# This function should return:
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix.

# This function should create a feature representation using all tokens that
# contain an alphabetic character.

def createBasicFeatures(corpus):
    # Extract texts and genres
    texts = [entry["text"] for entry in corpus]
    genres = [entry["genre"] for entry in corpus]

    # Step 1: Build vocabulary from all documents
    vocab_set = set()
    tokenized_docs = []

    for text in texts:
        tokens = tokenize(text)
        vocab_set.update(tokens)
        tokenized_docs.append(tokens)

    # Step 2: Create sorted vocabulary and word-to-index mapping
    vocab = sorted(list(vocab_set))
    word_to_idx = {word: idx for idx, word in enumerate(vocab)}

    # Step 3: Build sparse matrix
    row_indices = []
    col_indices = []
    data = []

    for doc_idx, tokens in enumerate(tokenized_docs):
        word_counts = Counter(tokens)
        for word, count in word_counts.items():
            if word in word_to_idx:
                row_indices.append(doc_idx)
                col_indices.append(word_to_idx[word])
                data.append(count)

    X = sparse.csr_matrix((data, (row_indices, col_indices)),
                          shape=(len(texts), len(vocab)))

    return X, genres, vocab

In [5]:
# TODO: Implement createInterestingFeatures. Describe your features and what
# they might tell you about the difference between comedy and tragedy.
# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.

"""
    Creates bigram features for classification.

    Features:
      - Only bigrams (sequences of two consecutive words).
      - Captures short phrases that may signal genre differences.
        For example:
            - Comedy might have bigrams like "funny joke", "haha laugh".
            - Tragedy might have bigrams like "death scene", "dark fate".

    Returns:
      - X: sparse matrix of bigram features
      - y: list of correct genres (labels)
      - vocab: list of bigram features in the vocabulary
  """

def createInterestingFeatures(corpus):
    texts = [entry["text"] for entry in corpus]
    genres = [entry["genre"] for entry in corpus]

    # Step 1: build vocabulary manually (only bigrams)
    vocab_counter = Counter()
    tokenized_docs = []

    for text in texts:
        tokens = tokenize(text)
        bigrams = [f"{tokens[i]} {tokens[i+1]}" for i in range(len(tokens)-1)]
        vocab_counter.update(bigrams)
        tokenized_docs.append(bigrams)

    # Step 2: build vocabulary (include all bigrams)
    vocab = list(vocab_counter.keys())
    vocab_index = {tok: i for i, tok in enumerate(vocab)}

    # Step 3: build sparse feature matrix
    row, col, data = [], [], []
    for doc_id, doc in enumerate(tokenized_docs):
        counts = Counter([tok for tok in doc if tok in vocab_index])
        for tok, cnt in counts.items():
            row.append(doc_id)
            col.append(vocab_index[tok])
            data.append(cnt)

    X = csr_matrix((data, (row, col)), shape=(len(texts), len(vocab)))

    return X, genres, vocab



In [6]:
print(createInterestingFeatures(readShakespeare()))

(<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 428922 stored elements and shape (26, 255076)>, ['comedy', 'tragedy', 'comedy', 'comedy', 'tragedy', 'tragedy', 'tragedy', 'tragedy', 'tragedy', 'comedy', 'tragedy', 'comedy', 'comedy', 'comedy', 'comedy', 'comedy', 'tragedy', 'tragedy', 'comedy', 'comedy', 'tragedy', 'tragedy', 'tragedy', 'comedy', 'comedy', 'comedy'], ['alls well', 'well that', 'that ends', 'ends well', 'well by', 'by william', 'william shakespeare', 'shakespeare dramatis', 'dramatis personae', 'personae king', 'king of', 'of france', 'france the', 'the duke', 'duke of', 'of florence', 'florence bertram', 'bertram count', 'count of', 'of rousillon', 'rousillon lafeu', 'lafeu an', 'an old', 'old lord', 'lord parolles', 'parolles a', 'a follower', 'follower of', 'of bertram', 'bertram two', 'two french', 'french lords', 'lords serving', 'serving with', 'with bertram', 'bertram steward', 'steward servant', 'servant to', 'to the', 'the countess', 'countess of',

In [7]:
#given a numpy matrix representation of the features for the training set, the
# vector of true classes for each example, and the vocabulary as described
# above, this computes the accuracy of the model using leave one out cross
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())

  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)

  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)


In [8]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [9]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.615385
The most informative terms for pos are: ['helena', 'prospero', 'duke', 'you', 'sir', 'i', 'leontes', 'a', 'of', 'your', 'presenteth', 'preserved', 'pretty', 'prettiness', 'prettily', 'prettiest', 'prettier', 'presentment', 'presently', 'presenting']
The most informative terms for neg are: ['him', 's', 'iago', 'imogen', 'brutus', 'lear', 'o', 'our', 'rom', 'what', 'ham', 'and', 'thy', 'the', 'prescription', 'pretense', 'pretending', 'pretended', 'presented', 'presentation']
----------L2 Norm-----------
The model's average accuracy is 0.769231
The most informative terms for pos are: ['i', 'you', 'duke', 'prospero', 'a', 'helena', 'your', 'antonio', 'sir', 'leontes', 'hermia', 'for', 'lysander', 'ariel', 'sebastian', 'demetrius', 'camillo', 'stephano', 'me', 'parolles']
The most informative terms for neg are: ['iago', 'othello', 's', 'him', 'imogen', 'what', 'lear', 'brutus', 'his', 'cassio', 'o', 'ham', 'our', 'desdemo

Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [10]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.730769
The most informative terms for pos are: ['not a', 'and i', 'me and', 'which is', 'the duke', 'she is', 'if you', 'you have', 'to me', 'i am', 'my love', 'the king', 'as you', 'sir toby', 'of syracuse', 'and so', 'all slops', 'bene gallants', 'no appearance', 'appearance of']
The most informative terms for neg are: ['the gods', 'art thou', 'the moor', 'give me', 'lady macbeth', 'my lord', 'to the', 'know not', 'in his', 'to thee', 'exeunt scene', 'caesar s', 'no doublet', 'strange disguises', 'disguises as', 'dutchman to', 'frenchman to', 'two countries', 'countries at', 'once as']
----------L2 Norm-----------
The model's average accuracy is 0.807692
The most informative terms for pos are: ['the duke', 'sir toby', 'i am', 'if you', 'and i', 'princess of', 'i pray', 'of france', 'sir i', 'she is', 'of syracuse', 'a good', 'in love', 'of a', 'to her', 'i will', 'the king', 'my love', 'antipholus of', 'you will']
The mos

**TODO**: Based on the most informative features in the output of the classifier evaluation, what do these classifiers tell you about the differences between comedy and tragedy?

**Interpretation of Classifier Features**

The classifier’s most informative features reveal clear differences between Shakespeare’s comedies and tragedies. In comedies, words like Helena, Prospero, Duke, and phrases such as “the duke”, “i am”, and “my love” are common. These features highlight characters engaged in social interactions, romance, and playful or clever situations, which are central to comedic plots. The language tends to be lighter, conversational, and relational, reflecting the humor, misunderstandings, and courtly dynamics that drive these stories.

In tragedies, by contrast, words like Iago, Ham, Brutus, and phrases such as “my lord”, “the gods”, and “lady macbeth” dominate. This points to more formal, serious dialogue, often dealing with authority, fate, or moral dilemmas. Tragedies use weighty, somber language, emphasizing tension, conflict, and consequence. Comparing L1 and L2 norms, L2 captures more frequent stylistic patterns, highlighting recurring structures in speech that distinguish the genres further.

Overall, the model shows that Shakespeare’s comedies and tragedies are not only defined by the characters they feature but also by the tone, style, and purpose of their dialogue, reflecting the different worlds these genres inhabit.